In [ ]:
#imported all the necessary libraries
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt 
import os
import sys
import cv2

In [ ]:
#defined an initial function for setting up the value of mean and variance using the first image 
def initBackground(initImage):
    img_arr = mpimg.imread(initImage)
    mean = img_arr
    variance = 9*np.ones(img_arr.shape) #taking 9 because pure square numbers give better results
    return(mean,variance)

In [ ]:
#taking parameters image, and previous mean and variance and lambda
def ForegroundDetection(img_file,mean,variance,lmda):
    img = cv2.imread(img_file) 
    d = img - mean
    y = variance*(lmda**2)
    d_2 = np.square(d)
    I = d_2 - y
    mask = np.all(I>0,axis=2) #taking axis 2 beacuse colored images have 3d matrix so channel is selected 2
    rI = 255*mask.astype(int) #if mask is true the pushed in r1 list and typecasted int 
    rI = rI.astype(np.uint8) #typecasted to unsigned int
    return(rI)

In [ ]:
#here we compare the neighbouring values of the target value to determine the state(dark or white) of the target
def Voting(rI,eta,m,n):
    r,c = rI.shape #just storing the shape
    cI = np.zeros((rI.shape[0],rI.shape[1]))  #defining  a matrix
    for i in range(m,r-1-m):
        for j in range(n,c-1-n):
            img_patch = rI[i-m:i,j-n:j]
            y_unq, counts = np.unique(img_patch,return_counts=True)
            # if the length f counts is greater than 1 the we do comarison betwee second value of counts and eta*m*n
            #and if the counts is greater than the pixel is white
            if len(counts) == 1 and y_unq[0] == 255:
                cI[i,j] = 255
            if len(counts)>1:
                if counts[1] > eta*m*n:
                    cI[i,j] = 255
    cI = cI.astype(np.uint8)
    return cI

In [ ]:
#here we update the values of mean and variance using the formula of background model update
def meanvarUpdate(cI,img_path,M,V,alpha):
    img = mpimg.imread(img_path)
    mean_upd = np.zeros(img.shape)
    var_upd = np.zeros(img.shape)
    d = img - M
    d_2 = np.square(d)
    for i in range(cI.shape[0]):
        for j in range(cI.shape[1]):
            if cI[i,j] == 0:
                mean_upd[i,j,:] = (1-alpha)*M[i,j,:] + alpha*img[i,j,:]
                var_upd[i,j,:] = (1-alpha)*(V[i,j,:] + alpha*d_2[i,j,:])
                var_upd[i,j,:] = np.clip(var_upd[i,j,:],a_min = 9,a_max = None)
    return(mean_upd,var_upd)

In [ ]:
#this is our master function where we call all the above defined functions
def Background_Subtraction(img_dir,lmda,eta,m,n,alpha):
    
    img_file_name = os.listdir(img_dir)
    sorted_img_file_name = sorted(img_file_name)
    initImage = os.path.join(img_dir,sorted_img_file_name[0])
    mean, variance = initBackground(initImage)
   #above four lines we take the first image and pass it to init function to calculate initial mean and variance 
    for i in range(1,19):
        img_path = os.path.join(img_dir,sorted_img_file_name[i])
        
        fig, ax = plt.subplots(1,3,figsize=(10,10))
        rI = ForegroundDetection(img_path,mean,variance,lmda)
        ax[0].imshow(rI,cmap="gray")
        
        cI = Voting(rI,eta,m,n)
        mean, variance = meanvarUpdate(cI,img_path,mean,variance,alpha)
        ax[1].imshow(cI,cmap="gray")
        
        img = cv2.imread(img_path)
        ax[2].imshow(img,cmap="gray")
        
        plt.show()
    #in the above four loop  we pass the raw image, filtered image and the colored image in imraw
    return(mean,variance)

In [ ]:
eta=0.7
lamda=0.8
for i in range(11):
    print('ETA = ',eta)
    print('LAMBDA = ',lamda)
    mean,variance = Background_Subtraction(".\AirStripVideoDataSetForBackGroundSubtraction\AirStripVideo\Images",lamda,eta,8,8,0.8)
    eta+=0.02
    print('##########################################################')

In [ ]:
eta=0.7
lamda=0.7
for i in range(11):
    print('ETA = ',eta)
    print('LAMBDA = ',lamda)
    mean,variance = Background_Subtraction(".\AirStripVideoDataSetForBackGroundSubtraction\AirStripVideo\Images",lamda,eta,8,8,0.8)
    lamda+=0.02
    print('#######################################################')